UI Demo

Can consider using Checkbutton to select if want to search by field, ascending/descending order etc

In [36]:
import tkinter as tk
import tkinter.ttk as ttk
import requests
from datetime import datetime

In [27]:
solr_url = 'http://localhost:8983/solr'
collection_name = 'restaurantCore'  

In [32]:
def query_by_category_name(category_name, num_results=20):
    # Define Solr query URL
    query_url = f'{solr_url}/{collection_name}/select'
    
    # Define Solr query parameters
    params = {
        'q': f'categoryName:"{category_name}"',  # Filter by CategoryName
        'group': 'true',  # Enable grouping
        'group.field': 'restaurant',  # Group by restaurant field to avoid duplicates
        'group.limit': 1,  # Limit to one document per group (top document for each restaurant)
        'rows': num_results,  # Limit number of results
        'debug': 'timing'  # Enable debug timing information
    }

    # Send GET request to Solr
    response = requests.get(query_url, params=params)

    # Check if request was successful (status code 200)
    if response.status_code == 200:
        # Parse JSON response
        results = response.json()
        grouped_results = results['grouped']['restaurant']['groups']
        
        # Get execution time from debug info
        debug_info = results['debug']
        execution_time = debug_info['timing']['time']

        return response.status_code, execution_time, grouped_results

    else:
        return response.status_code, "null", "null"

In [38]:
window = tk.Tk()
window.title("My Restaurant Search")

def display_query_results():
    query = entry.get()
    status, execution_time, grouped_results = query_by_category_name(query)
    if status != 200:
        # resp.configure(text=t)
        ttk.Label(window, text=f"Error: {status}").grid(row=1, column=0)
    else:
        out = ttk.Label(window, text=f"Top 20 results with CategoryName {query}")
        out.grid(row=2, column=0)
        out = ttk.Label(window, text=f"Execution Time: {execution_time} ms")
        out.grid(row=3, column=0)
        i = 3
        for group in grouped_results:
            restaurant_doc = group['doclist']['docs'][0]
            out = ttk.Label(window, text=f"Restaurant: {restaurant_doc['restaurant']}, categoryName: {restaurant_doc['categoryName']}")
            out.grid(row=i+1, column=0)
            i += 1

prompt = ttk.Label(window, text="Input your query here: ")
prompt.grid(row=0, column=0)

entry = tk.Entry(window, width=60)
entry.grid(row=0, column=1, padx=10, pady=10)

# resp = tk.Label(window, text="")
# resp.grid(row=1, column=0)

ttk.Button(window, text="Search", width=20, command=display_query_results).grid(row=0, column=2)
window.mainloop()